# Import packages

In [9]:
# Set auto reload for python functions
%reload_ext autoreload
%autoreload 2

# Import packages
import os
from time import time
from I_matrix import get_channel_sizes
from keras.applications.inception_v3 import preprocess_input

# GoogLeNet # Need to install lucid with the command: `pip install lucid`
import lucid.modelzoo.vision_models as models
import lucid.optvis.render as render

# Setup

## Hyperparameters and file paths

In [4]:
# GPU to use
use_gpu = False
gpu = 0

# Input directory path
input_dir_path = '/Users/haekyu/data/imagenet-tf-records'

# Output directory path
output_dir_path = '/Users/haekyu/data/massif/I-mat'

# Layer info
layer_sizes = {
    'mixed3a': 256,
    'mixed3b': 480,
    'mixed4a': 508,
    'mixed4b': 512,
    'mixed4c': 512,
    'mixed4d': 528,
    'mixed4e': 832,
    'mixed5a': 832,
    'mixed5b': 1024
}
layers = list(layer_sizes.keys())

# Hyperparameters

# Pick a layer
layer = 'mixed3a'

# Class info
num_class = 1000


In [5]:
# Assign a GPU
if use_gpu:
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)

## Import GoogLeNet

In [7]:
googlenet = models.InceptionV1()
googlenet.load_graphdef()
nodes = googlenet.graph_def.node

In [8]:
mixed_layers = [layer for layer in layers if 'mixed' in layer]
layer_fragment_sizes = {layer: get_channel_sizes(layer, nodes) for layer in mixed_layers}

# Get I-matrix

In [ ]:
def gen_I_matrix(layer, all_layers, input_dir_path):
    
    # Get previous layer
    prev_layer = get_prev_layer(all_layers, layer)

    # Time checker
    start_time = time()

    # Get file paths
    # file_paths = glob.glob('{}/*'.format(input_dir_path))
    file_paths = glob.glob('{}/train-00000-of-01024'.format(input_dir_path))

    # Get I-matrix
    with tf.Graph().as_default():

        # Define parsed dataset tensor
        dataset = tf.data.TFRecordDataset(filenames)
        dataset = dataset.map(_parse_function)
        dataset = dataset.map(lambda img, lab, syn: (preprocess_input(img), lab, syn))
        dataset = dataset.batch(batch)

        # Define iterator through the datasets
        iterator = dataset.make_one_shot_iterator()
        t_preprocessed_images, t_labels, t_synsets = iterator.get_next()

        # Define actiavtion map render
        T = render.import_model(googlenet, t_preprocessed_images, None)

        # Get weight tensors
        t_w0, t_w1, t_w2, t_w3, t_w4, t_w5 = get_weight_tensors(mixed_layer)

        # Get intermediate layer tensors
        t_a0, t_a1, t_a2 = get_intermediate_layer_tensors(prev_layer, mixed_layer)

        # Define intermediate conv output tensors
        t_inf_0 = get_infs(t_a0, t_w0)
        t_inf_1 = get_infs(t_a1, t_w2)
        t_inf_2 = get_infs(t_a2, t_w4)
        t_inf_3 = get_infs(t_a0, t_w5)
        t_inf_4 = get_infs(t_a0, t_w1)
        t_inf_5 = get_infs(t_a0, t_w3)

        # Run with batch
        progress_counter = 0
        with tf.Session() as sess:

            tic = time()

            try:
                with tqdm.tqdm(total=1281167, unit='imgs') as pbar:

                    while(True):
                        progress_counter += 1

                        # Run the session
                        if is_mixed:
                            labels, inf_0, inf_1, inf_2, inf_3 = sess.run([t_labels, t_inf_0, t_inf_1, t_inf_2, t_inf_3])

                        elif branch == 1:
                            labels, inf_4 = sess.run([t_labels, t_inf_4])

                        elif branch == 2:
                            labels, inf_5 = sess.run([t_labels, t_inf_5])

                        # Add up the counts
                        if is_mixed:
                            channel = 0
                            for frag, inf in enumerate([inf_0, inf_1, inf_2, inf_3]):
                                channel = update_I(layer, inf, channel, I_layer, labels, frag_sz[frag], k, outlier_nodes_idx)

                        elif branch == 1:
                            update_I(layer, inf_4, 0, I_layer, labels, frag_sz[4], k, outlier_nodes_idx)

                        elif branch == 2:
                            update_I(layer, inf_5, 0, I_layer, labels, frag_sz[5], k, outlier_nodes_idx)

                        pbar.update(len(labels))
                        # print(inf_0.shape, inf_1.shape, inf_2.shape, inf_3.shape, inf_4.shape, inf_5.shape)

            except tf.errors.OutOfRangeError:
                pass

            # Save I_layer
            with open(I_mat_dirpath + 'I_%s.json' % layer, 'w') as f:
                json.dump(I_layer, f, indent=2)

            end = time.time()
            print(end - start)
            print(progress_counter)
            print(progress_counter * batch)